In [12]:
!pip install ipykernel


In [7]:

!pip install "pinecone[grpc]"

In [1]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="pcsk_5bVfwr_FVWYHKchi2cYZCopXyb9JQreVW1hUfm97xcuAnamBkQtN31FXeGGpnfTZWnV1N8") # "YOUR_API_KEY"

# Create a dense index with integrated embedding
index_name = "reddit-shopper"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

pc.describe_index(name=index_name)

{
    "name": "reddit-shopper",
    "metric": "cosine",
    "host": "reddit-shopper-9l7icfb.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null,
    "embed": {
        "model": "llama-text-embed-v2",
        "field_map": {
            "text": "chunk_text"
        },
        "dimension": 1024,
        "metric": "cosine",
        "write_parameters": {
            "dimension": 1024.0,
            "input_type": "passage",
            "truncate": "END"
        },
        "read_parameters": {
            "dimension": 1024.0,
            "input_type": "query",
            "truncate": "END"
        },
        "vector_type": "dense"
    }
}

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("reddit_comments.csv")

In [4]:
df

,Post Title,Post URL,Comment
0,This Landscape is Beautiful,https://www.reddit.com/r/Outdoors/comments/1ji...,"Stäubi Falls, Switzerland."
1,This Landscape is Beautiful,https://www.reddit.com/r/Outdoors/comments/1ji...,Every beautiful clip shorter than 10 seconds i...
2,This Landscape is Beautiful,https://www.reddit.com/r/Outdoors/comments/1ji...,didn't even need the comments to know I was lo...
3,This Landscape is Beautiful,https://www.reddit.com/r/Outdoors/comments/1ji...,You couldnt pay me enough to not live there.
4,This Landscape is Beautiful,https://www.reddit.com/r/Outdoors/comments/1ji...,Looks like portrait to me. lol
...,...,...,...
1039,Wouldn’t mind spending a season at this fire l...,https://www.reddit.com/r/Outdoors/comments/1jh...,Reminds me of Days Gone. Beautiful view
1040,Wouldn’t mind spending a season at this fire l...,https://www.reddit.com/r/Outdoors/comments/1jh...,How do you apply????
1041,"A little piece of Bariloche in Patagonia, one ...",https://www.reddit.com/r/Outdoors/comments/1j6...,Are those skylights in the grass? Is there a e...
1042,"A little piece of Bariloche in Patagonia, one ...",https://www.reddit.com/r/Outdoors/comments/1j6...,One of my favorite places on earth.


In [6]:
df['_id'] = 'rec' + df.index.astype(str)

records = (
    df[['_id', 'Post Title', 'Comment']]
    .rename(columns={'Comment': 'chunk_text', 'Post Title': 'category'})
    .to_dict(orient='records')
)

records

[{'_id': 'rec0',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'Stäubi Falls, Switzerland.'},
 {'_id': 'rec1',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'Every beautiful clip shorter than 10 seconds is suspicious regarding being AI-generated. At least in my head. ;)'},
 {'_id': 'rec2',
  'category': 'This Landscape is Beautiful',
  'chunk_text': "didn't even need the comments to know I was looking at Switzerland. that place is gorgeous x3"},
 {'_id': 'rec3',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'You couldnt pay me enough to not live there.'},
 {'_id': 'rec4',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'Looks like portrait to me. lol'},
 {'_id': 'rec5',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'brb imagining a sunday morning hike here'},
 {'_id': 'rec6',
  'category': 'This Landscape is Beautiful',
  'chunk_text': 'Everytime i see switzerland i think how lovely it would be to build a house th

In [7]:
reddit_shopper = pc.Index(index_name)

for rec in records:
    try:
        reddit_shopper.upsert_records("reddit-namespace", [rec])
        print(f"Record {rec['_id']} upserted successfully.")
    except Exception as e:
        print(f"Error upserting record {rec['_id']}: {e}")

Record rec0 upserted successfully.
Record rec1 upserted successfully.
Record rec2 upserted successfully.
Record rec3 upserted successfully.
Record rec4 upserted successfully.
Record rec5 upserted successfully.
Record rec6 upserted successfully.
Record rec7 upserted successfully.
Record rec8 upserted successfully.
Record rec9 upserted successfully.
Record rec10 upserted successfully.
Record rec11 upserted successfully.
Record rec12 upserted successfully.
Record rec13 upserted successfully.
Record rec14 upserted successfully.
Record rec15 upserted successfully.
Record rec16 upserted successfully.
Record rec17 upserted successfully.
Record rec18 upserted successfully.
Record rec19 upserted successfully.
Record rec20 upserted successfully.
Record rec21 upserted successfully.
Record rec22 upserted successfully.
Record rec23 upserted successfully.
Record rec24 upserted successfully.
Record rec25 upserted successfully.
Record rec26 upserted successfully.
Record rec27 upserted successfully.
Re